In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

First we will simulate a pickup and delivery problem for our food delivery company. First we set up the coordinates of restaurants (pickup locations) and households (delivery locations) as follows. Every spot is a vertex on a grid where dimension of each unit is 114 * 80.

[ (456, 320), # location 0 - depot <br>
(228, 0),     # location 1 <br>
(912, 0),     # location 2 <br>
(0, 80),      # location 3 <br>
(114, 80),    # location 4 <br>
(570, 160),   # location 5 <br>
(798, 160),   # location 6 <br>
(342, 240),   # location 7 <br>
(684, 240),   # location 8 <br>
(570, 400),   # location 9 <br>
(912, 400),   # location 10 <br>
(114, 480),   # location 11 <br>
(228, 480),   # location 12 <br>
(342, 560),   # location 13 <br>
(684, 560),   # location 14 <br> 
(0, 640),     # location 15 <br>
(798, 640)].  # location 16

For the sake of a better simulation in setting up the problem, we will be using Manhattan distance to calculate distances between locations, since road structure of a city resembles a grid-like system. Manhattan distance is defined as the distance between two points (x_1, y_1) and (x_2, y_2) being |x_1 - x_2| + |y_1 - y_2|. Now we sets up the distance matrix and a list of pairs of pickup and delivery locations. 

In [53]:
def create_data_model():
    """
    Stores the data for the problem.
    """
    
    data = {}
    
    #distance matrix by Manhattan distance
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    
    #pairs of pickup and delivery locations, first entry is index of pickup while second entry is index of delivery
    data['pickups_deliveries'] = [
        [1, 6], 
        [2, 10], 
        [4, 3], 
        [5, 9], 
        [7, 8], 
        [15, 11], 
        [13, 12], 
        [16, 14]
    ]
    
    #preset number of vehicles used in the problem
    data['num_vehicles'] = 6
    
    #sets index of depot location to be 0
    data['depot'] = 0
    
    return data

In [41]:
def print_solution(data, manager, routing, solution):
    """
    Prints the optimized route in console.
    """
        
    print(f'Objective: {solution.ObjectiveValue()}')
    #tracks total distance for all vehicles
    total_route_distance = 0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}\n'.format(route_distance)
        print(plan_output)
        total_route_distance += route_distance
    
    print('Total distance of all routes: {}'.format(total_route_distance))

In [54]:
def main():
    """
    Entry point of the program.
    """
    
    #instantiates the data
    data = create_data_model()
    
    #creates the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), 
                                          data['num_vehicles'], data['depot'])
    
    #initializes routing model
    routing = pywrapcp.RoutingModel(manager)
    
    #creates a transit callback
    def distance_callback(from_index, to_index):
        """
        Returns the manhattan distance between two nodes.
        """
        
        #converts from routing variable index to distance matrix NodeIndex using manager
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        
        return data['distance_matrix'][from_node][to_node]
    
    #defines cost of each arc
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    #adds distance dimension
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index, 
        0, 
        3000, # vehicle maximum travel distance
        True, # start cumulative to 0
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    #defines pickup and delivery request
    for request in data['pickups_deliveries']:
        #first entry being pickup, second entry being delivery
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        
        #makes sure that one vehicle serves each pair of pickup and delivery node
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
        
        #makes sure that the vehicle reached pickup node before delivery node for each pair
        routing.solver().Add(distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index))
    
    #trys different algorithms to reach optimized route
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)  # insertion heuristic
    
#     search_parameters = pywrapcp.DefaultRoutingSearchParameters()
#     search_parameters.first_solution_strategy = (
#         routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)  # classic heuristic
    
#     search_parameters = pywrapcp.DefaultRoutingSearchParameters()
#     search_parametsrs.first_solution_strategy = (
#         routing_enums_pb2.FirstSolutionStrategy.SAVINGS)  # savings
    
#     search_parameters = pywrapcp.DefaultRoutingSearchParameters()
#     search_parametsrs.first_solution_strategy = (
#         routing_enums_pb2.FirstSolutionStrategy.SWEEP)  # sweep

    #implements optimization
    solution = routing.SolveWithParameters(search_parameters)
    
    #prints solution on console
    if solution:
        print_solution(data, manager, routing, solution)

if __name__ == '__main__':
    main()

Objective: 185760
Route for vehicle 0:
 0 ->  13 ->  15 ->  11 ->  12 -> 0
Distance of the route: 1552

Route for vehicle 1:
 0 ->  16 ->  14 -> 0
Distance of the route: 1324

Route for vehicle 2:
 0 ->  4 ->  3 -> 0
Distance of the route: 1392

Route for vehicle 3:
 0 ->  7 ->  1 ->  6 ->  8 -> 0
Distance of the route: 1780

Route for vehicle 4:
 0 ->  5 ->  2 ->  10 ->  9 -> 0
Distance of the route: 1712

Route for vehicle 5:
 0 -> 0
Distance of the route: 0

Total distance of all routes: 7760
